In [379]:
import re, math
import operator
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
from random import randint
from nltk.corpus import stopwords
from nltk import TweetTokenizer
import sys, os
global stop_words
stop_words = stopwords.words('english')
import datetime
from matplotlib import dates
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.dates as mdates
from ipywidgets import IntProgress
from IPython.display import display
import string
sys.path.append('../English_to_IPA/src/')
import conversion as cv
sys.path.append('../age_gender_predictor/')
import predictor
import copy

In [35]:
class Patient(object):
    totalCount = 0
    
    def __init__(self, userId):
        self.userId = userId
        self.df = pd.read_csv('../0_dataset/DepressionUsersTweets/Patch0_data_PoL/'+userId,sep='\t',header=None,usecols=[0,1,2,3,4,5,6,7],names=['Date','Id','Name','Text','Senti','Emo1','Emo2','Ambiguos'],quoting=3,error_bad_lines=False,encoding='utf-8').dropna(axis=0, how='any')
        self.df = self.df.drop_duplicates(subset='Date', keep='first')
        self.df = self.df.set_index('Date')
        self.df.index = pd.to_datetime(self.df.index)
        
        Patient.totalCount += 1 
        
    def displayCount(self):
        print("Total Patients {0}".format(Patient.totalCount))
    
    def displayTweetsCount(self):
        return len(self.df)
        
    def getText(self):
        return self.df['Text'].values

In [71]:
class Ordinary(object):
    totalCount = 0
    
       
    def __init__(self, userId):
        self.userId = userId
        self.df = pd.read_csv('../0_dataset/OrdinaryUsersTweets_PoL/'+userId,sep='\t',header=None,usecols=[0,1,2,3,4,5,6,7],names=['Date','Id','Name','Text','Senti','Emo1','Emo2','Ambiguos'],quoting=3,error_bad_lines=False,encoding='utf-8').dropna(axis=0, how='any')
        self.df = self.df.drop_duplicates(subset='Date', keep='first')
        self.df = self.df.set_index('Date')
        self.df.index = pd.to_datetime(self.df.index)
        self.df = self.df.sort_index(ascending=False)
        
        Patient.totalCount += 1 
        
    def displayCount(self):
        print("Total Patients {0}".format(Patient.totalCount))
    
    def displayTweetsCount(self):
        return len(self.df)
       
        
    def getText(self):
        return self.df['Text'].values

In [17]:
def findUserIdFromDataset(group, num):
    userIds = []
    print(group+str(num))
    with open('../0_dataset/' + group + 'Dataset' + str(num)) as open_file:
        for line in open_file.readlines():
            userIds.append(line.strip().split('\t')[0])
            
    return userIds

In [22]:
def readID(filename):
    ids = []
    with open(filename) as openFile:
        for line in openFile.readlines():
            ids.append(line.strip())
            
    return ids

In [238]:
def tokenize(text):
    textList = re.findall('(?u)\\b[a-zA-Z]\\w{0,}\\b', text)
    return textList

def replace_by_symbols(txt):
    txt = re.sub(r"https\S+", '', txt)
    txt = re.sub(r"http\S+", '', txt)
    txt = re.sub(r"pic.twitter.com\S+", '', txt)
    txt = re.sub(r"twitter.com/\S+", '', txt)
    txt = re.sub(r"\S+/\S+", '', txt)
    txt = re.sub(r"@\S+", '', txt)
    txt = re.sub(r"#\S+", '', txt)
    txt = re.sub(r"idk", 'i do not know', txt)   # idk: i don't know
    txt = re.sub(r"tbh", 'to be honest', txt)   # tbh: to be honest
    txt = re.sub(r"tho", 'though', txt)   # tho
    txt = re.sub(r"i\'m", 'i am', txt)
    txt = re.sub(r"you\'re", 'you are', txt)
    txt = re.sub(r"he\'s", 'he is', txt)
    txt = re.sub(r"she\'s", 'she is', txt)
    txt = re.sub(r"it\'s", 'it is', txt)
    txt = re.sub(r"we\'re", 'we are', txt)
    txt = re.sub(r"they\'re", 'they are', txt)
    txt = re.sub(r"isn\'t", 'is not', txt)
    txt = re.sub(r"don\'t", 'do not', txt)
    txt = re.sub(r"doesn\'t", 'does not', txt)
    txt = re.sub(r"didn\'t", 'did not', txt)
    txt = re.sub(r"wasn\'t", 'was not', txt)
    txt = re.sub(r"weren\'t", 'were not', txt)
    txt = re.sub(r"haven\'t", 'have not', txt)
    txt = re.sub(r"can\'t", 'can not', txt)
    txt = re.sub(r"couldn\'t", 'could not', txt)
    txt = re.sub(r"wouldn\'t", 'would not', txt)
    txt = re.sub(r"shouldn\'t", 'should not', txt)
    txt = re.sub(r"&amp", '', txt)
    return txt

# Read UserID

In [19]:
GD1, GD2, GD3 = ([] for i in range(3))
BD1, BD2, BD3 = ([] for i in range(3))

In [63]:
GD1 = readID('../0_dataset/patientDataset1ID')
GD2 = readID('../0_dataset/patientDataset2ID')
GD3 = readID('../0_dataset/patientDataset3ID')
BD1 = readID('../0_dataset/ordinaryDataset1ID')
BD2 = readID('../0_dataset/ordinaryDataset2ID')
BD3 = readID('../0_dataset/ordinaryDataset3ID')

In [64]:
%%time
patientsD1 = dict()
patientsD2 = dict()
patientsD3 = dict()

for ele in GD1:
    patientsD1[ele] = Patient(ele)
for ele in GD2:
    patientsD2[ele] = Patient(ele)
for ele in GD3:
    patientsD3[ele] = Patient(ele)

CPU times: user 2.92 s, sys: 28 ms, total: 2.95 s
Wall time: 3.3 s


In [72]:
%%time
ordinarysD1 = dict()
ordinarysD2 = dict()
ordinarysD3 = dict()

for ele in BD1:
    ordinarysD1[ele] = Ordinary(ele)
for ele in BD2:
    ordinarysD2[ele] = Ordinary(ele)
for ele in BD3:
    ordinarysD3[ele] = Ordinary(ele)

CPU times: user 1.93 s, sys: 16 ms, total: 1.94 s
Wall time: 2.81 s


# *** eSPE V2 ***

In [247]:
eSPEDictV2 = dict()
with open('eSPEPhonologicalTableV2') as openFile:
    for line in openFile.readlines():
        line = line.strip().split('\t')
        eSPEDictV2[line[0]] = np.array([eval(x) for x in line[1:]])
        
def getTweetScoreV2(tweet):
    wordCount = 0
    words = tokenize(tweet)
    score = np.zeros((8,), dtype=np.int)
    for word in words:
        # 字典裡找不到這個字
        cmu, ipa = cv.convert(word)
        ipa = re.sub('[ˌˈ ]' ,'' ,ipa)
        if '*' in ipa:
            continue
        wordCount += 1
        i = 0
        while i < len(ipa):
            # 最後一個音標
            if i == len(ipa)-1:
                sym = ipa[i]
                score += eSPEDictV2[sym]
                i += 1
            # 非最後一個音標
            else:
                try:
                    sym = ipa[i] + ipa[i+1]
                    score += eSPEDictV2[sym]
                    i += 2
                except KeyError:
                    sym = ipa[i]
                    score += eSPEDictV2[sym]
                    i += 1

    score = np.append(score, wordCount)
    return score

# Input:  該群人的文字
# Return: 該群人的聲音分數
def getPhonologicalScores(groupText):
    groupScoresV2 = []
    for i, person in enumerate(groupText):
        tweets = person.split('\n')
        score = np.zeros((9,), dtype=np.int)
        for tweet in tweets:
            score += getTweetScoreV2(tweet)
        groupScoresV2.append(list(score))
    
    return groupScoresV2

In [402]:
def standardizeFeatures(featuresList):
    ''' 調整原始特徵數（以probability取代quntity）
        Parameter (textList): 該群使用者的特徵 List
        Return: 調整過後的 List
    '''
    
    total = []
    
    for features in featuresList:
        standardizedFeatures = []
        length = len(features)
        for i, ele in enumerate(features):
            if i != (length-1):
                if features[-1] == 0:
                    standardizedFeatures.append(0)
                else:
                    standardizedFeatures.append(round(ele/features[-1],2))
            else:
                if features[-1] == 0:
                    standardizedFeatures.append(0)
                else:
                    standardizedFeatures.append(features[-1])
        total.append(standardizedFeatures[0:-1])
    return total

# *** Pattern of Life ***

In [109]:
tweetTknzr = TweetTokenizer()

# Daily Tweet Frequency

In [299]:
def getTweetRate(df):
    totalTweets = len(df)
    delta_time = np.max(df.index.values) - np.min(df.index.values)
    totla_duration = (delta_time).astype('timedelta64[h]') / np.timedelta64(24, 'h')
    try:
        return totalTweets / float(totla_duration)
    except ZeroDivisionError:
        return 0

# Mention Ratio

In [121]:
def thirdPronuonDetect(words, matcher=re.compile("@[a-z]+")):
    for word in words:
        if word == "@":
            continue
        elif matcher.search(word):
            return True
    return False

In [266]:
def getMentionRate(df):
    ''' 發文提到別人的比例 '''
    
    totalTweets = len(df)
    total_mentions = 0
    for item in df.iterrows():
        if thirdPronuonDetect(tweetTknzr.tokenize(item[1]['Text'])):
            total_mentions += 1

    return total_mentions/float(totalTweets)

# Frequent Mentions

In [123]:
def getFrequentMetions(df, lowerbound=3):
    friends_mentions = {}
    for item in df.iterrows():
        for word in tweetTknzr.tokenize(item[1]['Text']):
            if word[0] == '@' and len(word) > 1:
                friends_mentions[word] = friends_mentions.get(word, 0) +1
    frequent_frients = [screen_name for screen_name, mentions in friends_mentions.items() if mentions >= lowerbound]

    return len(frequent_frients)

# Unique Mentions

In [127]:
def getUniqueMentions(df):
    totalTweets = len(df)
    friends_set = set()
    for item in df.iterrows():
        for word in tweetTknzr.tokenize(item[1]['Text']):
            if word[0] == '@' and len(word) > 1:
                friends_set.add(word)
                
    return len(friends_set)

# Gender / Age Query Function

In [267]:
def getAge(df):
    # This function returns a float, representing the age. 
    texts = ""
    for item in df.iterrows():
        texts += item[1]['Text'] + "\n"
    return predictor.get_age(texts)

def getGender(df):
    # This function returns a float. Positive valuse represents female and vice versa.
    texts = ""
    for text in df.iterrows():
        texts += item[1]['Text'] + "\n"
    return predictor.get_gender(texts)

# Emotion Scores

In [268]:
def getEmotionsDict(df):
    user = df['Id'][0]
    tweetsCount = 0
    emotionDict = {"joy":0,"sadness": 0,"fear":0,"anticipation": 0,"anger":0,"trust": 0,"disgust": 0,"surprise" : 0}
    for item in df.iterrows():
        if item[1]['Ambiguos'] != 'yes':
            tweetsCount += 1
            emotion = item[1]['Emo1']
            emotionDict[emotion] += 1
    
    for emotion in emotionDict:
            if tweetsCount == 0:
                emotionDict[emotion] = 0
            else:
                emotionDict[emotion] = float(emotionDict[emotion]) / tweetsCount
    
    return emotionDict

# Positive / Negative Ratio

In [269]:
def getNegativeRatio(df):
    totalTweets = len(df)
    return sum(df.Senti == -1) / float(totalTweets)

In [270]:
def getPositiveRatio(df):
    totalTweets = len(df)
    return sum(df.Senti == 1) / float(totalTweets)

# Get Pattern of Life

In [272]:
def getPatternOfLifeFeatures(df):
    score = np.zeros((17,), dtype=np.float)
    score[0] = round(getTweetRate(df), 2)
    score[1] = round(getMentionRate(df), 2)
    score[2] = getFrequentMetions(df)
    score[3] = getUniqueMentions(df)
    score[4] = round(getAge(df), 1)
    if getGender(df) >= 0:
        score[5] = 1
    else:
        score[6] = 0
    
    emotionsDict = getEmotionsDict(df)
    score[7] = round(emotionsDict["joy"], 2)
    score[8] = round(emotionsDict["sadness"], 2)
    score[9] = round(emotionsDict["fear"], 2)
    score[10] = round(emotionsDict["anticipation"], 2)
    score[11] = round(emotionsDict["anger"], 2)
    score[12] = round(emotionsDict["trust"], 2)
    score[13] = round(emotionsDict["disgust"], 2)
    score[14] = round(emotionsDict["surprise"], 2)
    score[15] = round(getPositiveRatio(df), 2)
    score[16] = round(getNegativeRatio(df), 2)
    
    return list(score)

# Playgound Here

In [107]:
test = ordinarysD1['1007626946'].df

ValueError: operands could not be broadcast together with shapes (17,) (9,) 

In [387]:
list(groupD1PoL)

[array([  1.21300000e+01,   1.00000000e-02,   0.00000000e+00,
          5.00000000e+00,   2.12000000e+01,   1.00000000e+00,
          0.00000000e+00,   7.00000000e-02,   5.40000000e-01,
          8.00000000e-02,   5.00000000e-02,   1.30000000e-01,
          2.00000000e-02,   9.00000000e-02,   3.00000000e-02,
          4.00000000e-02,   4.00000000e-01]),
 array([  2.27000000e+00,   1.00000000e-02,   4.00000000e+00,
          7.00000000e+00,   1.66000000e+01,   1.00000000e+00,
          0.00000000e+00,   1.50000000e-01,   3.70000000e-01,
          1.00000000e-02,   1.80000000e-01,   1.10000000e-01,
          6.00000000e-02,   9.00000000e-02,   3.00000000e-02,
          1.20000000e-01,   2.20000000e-01]),
 array([  0.42,   0.29,   1.  ,   8.  ,  13.5 ,   1.  ,   0.  ,   0.  ,
          0.45,   0.  ,   0.18,   0.18,   0.09,   0.09,   0.  ,   0.12,
          0.06]),
 array([  1.88000000e+00,   1.40000000e-01,   2.00000000e+00,
          4.70000000e+01,   2.26000000e+01,   1.00000000e+00,
  

# *** Extracting Feature ***

In [388]:
%%time
groupD1Text, groupD1PoL, groupD1Sound, groupD1SoundStd = ([] for i in range(4))
for patient in patientsD1:
    groupD1Text.append(replace_by_symbols('\n'.join(patientsD1[patient].getText())))
    groupD1PoL.append(getPatternOfLifeFeatures(patientsD1[patient].df))
groupD1Sound = getPhonologicalScores(groupD1Text)
groupD1SoundStd = standardizeFeatures(groupD1Sound)

    
groupD2Text, groupD2PoL, groupD2Sound, groupD2SoundStd = ([] for i in range(4))
for patient in patientsD2:
    groupD2Text.append(replace_by_symbols('\n'.join(patientsD2[patient].getText())))
    groupD2PoL.append(getPatternOfLifeFeatures(patientsD2[patient].df))
groupD2Sound = getPhonologicalScores(groupD2Text)
groupD2SoundStd = standardizeFeatures(groupD2Sound)


groupD3Text, groupD3PoL, groupD3Sound, groupD3SoundStd = ([] for i in range(4))
for patient in patientsD3:
    groupD3Text.append(replace_by_symbols('\n'.join(patientsD3[patient].getText())))
    groupD3PoL.append(getPatternOfLifeFeatures(patientsD3[patient].df))
groupD3Sound = getPhonologicalScores(groupD3Text)
groupD3SoundStd = standardizeFeatures(groupD3Sound)

CPU times: user 7min 57s, sys: 1.36 s, total: 7min 58s
Wall time: 7min 54s


In [389]:
%%time
baseD1Text, baseD1PoL, baseD1Sound, baseD1SoundStd = ([] for i in range(4))
for ordinary in ordinarysD1:
    baseD1Text.append(replace_by_symbols('\n'.join(ordinarysD1[ordinary].getText())))
    baseD1PoL.append(getPatternOfLifeFeatures(ordinarysD1[ordinary].df))
baseD1Sound = getPhonologicalScores(baseD1Text)
baseD1SoundStd = standardizeFeatures(baseD1Sound)   

baseD2Text, baseD2PoL, baseD2Sound, baseD2SoundStd = ([] for i in range(4))
for ordinary in ordinarysD2:
    baseD2Text.append(replace_by_symbols('\n'.join(ordinarysD2[ordinary].getText())))
    baseD2PoL.append(getPatternOfLifeFeatures(ordinarysD2[ordinary].df))
baseD2Sound = getPhonologicalScores(baseD2Text)
baseD2SoundStd = standardizeFeatures(baseD2Sound)

baseD3Text, baseD3PoL, baseD3Sound, baseD3SoundStd = ([] for i in range(4))
for ordinary in ordinarysD3:
    baseD3Text.append(replace_by_symbols('\n'.join(ordinarysD3[ordinary].getText())))
    baseD3PoL.append(getPatternOfLifeFeatures(ordinarysD3[ordinary].df))
baseD3Sound = getPhonologicalScores(baseD3Text)
baseD3SoundStd = standardizeFeatures(baseD3Sound)

CPU times: user 1min 4s, sys: 216 ms, total: 1min 5s
Wall time: 1min 4s


# Combine PoL and Sound

In [403]:
def combinePoLandSound(listPoL, listSound):
    listPoLandSound = []
    length = len(listPoL)
    for i in range(length):
        tmp = copy.deepcopy(list(listPoL[i]))
        tmp.extend(listSound[i][0:-1])    # [0:-1]沒有考慮字數
        listPoLandSound.append(tmp)
    
    return listPoLandSound

In [404]:
groupD1PoLAndSound, groupD2PoLAndSound, groupD3PoLAndSound = ([] for i in range(3))
baseD1PoLAndSound, baseD2PoLAndSound, baseD3PoLAndSound = ([] for i in range(3))

In [405]:
groupD1PoLAndSound = combinePoLandSound(groupD1PoL, groupD1SoundStd)
groupD2PoLAndSound = combinePoLandSound(groupD2PoL, groupD2SoundStd)
groupD3PoLAndSound = combinePoLandSound(groupD3PoL, groupD3SoundStd)
baseD1PoLAndSound = combinePoLandSound(baseD1PoL, baseD1SoundStd)
baseD2PoLAndSound = combinePoLandSound(baseD2PoL, baseD2SoundStd)
baseD3PoLAndSound = combinePoLandSound(baseD3PoL, baseD3SoundStd)

# Store Classifier Information

In [411]:
class ClassfierInfo(object):
    
    def __init__(self, name, cl, X, Y):
        self.name = name
        self.trainIndex = list()
        self.testIndex = list()
        self.precision = 0
        self.X = X
        self.Y = Y
        self.classifier = cl
#         self.prediction, self.bias, self.contributions = .predict(cl, X)

# RandomForest Classifier

In [412]:
def getRandomForestPrecision(baseFeatures, groupFeatures, storageName, fold=10):
    X = np.array(baseFeatures + groupFeatures)
    Y = np.array([0] * len(baseFeatures) + [1]*len(groupFeatures), dtype=int)
    sss = StratifiedShuffleSplit(Y, fold, random_state=randint(0,65536))
    classifier  = RandomForestClassifier(n_jobs=-1, max_features="sqrt", n_estimators=128)

    classifierStorage = [None]*(sss.n_iter)
    i = 0
    for train_index, test_index in sss:

        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        classifier = RandomForestClassifier(n_jobs=-1, max_features="sqrt", n_estimators=128)
        classifier.fit(X_train, Y_train)
        precision = classifier.score(X_test, Y_test)

        classifierStorage[i] = ClassfierInfo(storageName+"-"+str(i), classifier, X, Y)
        classifierStorage[i].train_index = train_index
        classifierStorage[i].test_index = test_index
        classifierStorage[i].precision = precision
        i += 1

    avgPrecision = 0
    print("Precision of {0}".format(storageName))
    print("-"*30)
    for i, cl in enumerate(classifierStorage):
        print("{0}\t{1}".format(i, cl.precision))
        avgPrecision += cl.precision
    print("-"*30)    
    print("Avg. precision: {0}".format(avgPrecision/fold))
    
    
    return classifierStorage

In [413]:
classfierD1 = getRandomForestPrecision(baseD1PoLAndSound, groupD1PoLAndSound, 'D1')

Precision of D1
------------------------------
0	1.0
1	0.9333333333333333
2	0.9666666666666667
3	0.9666666666666667
4	0.9666666666666667
5	1.0
6	1.0
7	1.0
8	1.0
9	1.0
------------------------------
Avg. precision: 0.9833333333333332


In [435]:
classfierD1[0].classifier.feature_importances_

array([ 0.08774923,  0.00596148,  0.00602803,  0.0064884 ,  0.01729932,
        0.        ,  0.        ,  0.0036945 ,  0.00559356,  0.01893263,
        0.00921569,  0.01037329,  0.00446645,  0.00715401,  0.10923277,
        0.23272461,  0.35836157,  0.01930981,  0.01123673,  0.01988788,
        0.01901485,  0.0054097 ,  0.00835247,  0.0260299 ,  0.00748311])

In [418]:
classfierD1PoL = getRandomForestPrecision(baseD1PoL, groupD1PoL, 'D1-PoL')

Precision of D1-PoL
------------------------------
0	1.0
1	1.0
2	1.0
3	0.9666666666666667
4	0.9666666666666667
5	1.0
6	1.0
7	0.9666666666666667
8	0.9333333333333333
9	0.9666666666666667
------------------------------
Avg. precision: 0.9800000000000001


In [436]:
classfierD1PoL[0].classifier.feature_importances_

array([ 0.06888453,  0.00595246,  0.00937269,  0.01138908,  0.03581087,
        0.        ,  0.        ,  0.00916809,  0.01834793,  0.01455097,
        0.00836097,  0.01771271,  0.01157389,  0.01815934,  0.09358505,
        0.24844239,  0.42868902])

In [416]:
classfierD2 = getRandomForestPrecision(baseD2PoLAndSound, groupD2PoLAndSound, 'D2')

Precision of D2
------------------------------
0	1.0
1	0.9666666666666667
2	1.0
3	1.0
4	0.9666666666666667
5	0.9666666666666667
6	1.0
7	1.0
8	0.9
9	1.0
------------------------------
Avg. precision: 0.9800000000000001


In [437]:
classfierD2[0].classifier.feature_importances_

array([ 0.07412156,  0.00520173,  0.00279475,  0.00872906,  0.01723311,
        0.        ,  0.        ,  0.00551009,  0.00652568,  0.00781666,
        0.01288525,  0.01753142,  0.00581413,  0.00587582,  0.10877502,
        0.28402855,  0.30688402,  0.01206548,  0.01450654,  0.02831948,
        0.03224837,  0.00494498,  0.00958443,  0.01986552,  0.00873834])

In [417]:
classfierD2PoL = getRandomForestPrecision(baseD2PoL, groupD2PoL, 'D2-PoL')

Precision of D2-PoL
------------------------------
0	0.9666666666666667
1	1.0
2	0.9666666666666667
3	0.9666666666666667
4	0.9666666666666667
5	1.0
6	0.9666666666666667
7	0.9666666666666667
8	1.0
9	0.9666666666666667
------------------------------
Avg. precision: 0.9766666666666668


In [438]:
classfierD2PoL[0].classifier.feature_importances_

array([ 0.08961264,  0.00687797,  0.00681916,  0.01025267,  0.03044665,
        0.        ,  0.        ,  0.00841392,  0.0174694 ,  0.01465152,
        0.01328498,  0.021413  ,  0.01082464,  0.01933149,  0.11995047,
        0.24802527,  0.38262621])

In [431]:
classfierD3 = getRandomForestPrecision(baseD3PoLAndSound, groupD3PoLAndSound, 'D3')

Precision of D3
------------------------------
0	1.0
1	0.9666666666666667
2	1.0
3	1.0
4	1.0
5	1.0
6	1.0
7	1.0
8	1.0
9	0.9666666666666667
------------------------------
Avg. precision: 0.9933333333333334


In [439]:
classfierD3[0].classifier.feature_importances_

array([ 0.08454541,  0.00795675,  0.00901815,  0.00994495,  0.01473869,
        0.        ,  0.        ,  0.00445123,  0.00456905,  0.01577546,
        0.01179244,  0.02742571,  0.00369138,  0.0074923 ,  0.08334879,
        0.29086979,  0.32897338,  0.01400976,  0.01122154,  0.0169663 ,
        0.01341126,  0.00619417,  0.00362518,  0.01853572,  0.0114426 ])

In [433]:
classfierD3PoL = getRandomForestPrecision(baseD3PoL, groupD3PoL, 'D3-PoL')

Precision of D3-PoL
------------------------------
0	1.0
1	1.0
2	0.9666666666666667
3	1.0
4	1.0
5	0.9666666666666667
6	1.0
7	0.9666666666666667
8	1.0
9	0.9666666666666667
------------------------------
Avg. precision: 0.9866666666666667


In [440]:
classfierD3PoL[0].classifier.feature_importances_

array([ 0.09239525,  0.00733732,  0.01195268,  0.01867531,  0.03212464,
        0.        ,  0.        ,  0.0045369 ,  0.00646549,  0.00632691,
        0.01951592,  0.02581771,  0.00971273,  0.01297086,  0.05282221,
        0.26651012,  0.43283596])